# Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados AVI SSVEP simplificado para um participante.

### Passos para a realização da classificação:

1. Carrega o arquivo fif (mne.EpochsArray);
2. Obter a "energia" do sinal por meio do cálculo `compute_psd` para os dados no domínio do tempo;
4. Com os dados PSD, encontre o valor máximo para cada _trial_ e compare com o valor que está sendo evocado.
5. Por fim, acumule os acertos para mostrar uma acurácia simples.

In [1]:
import mne
import numpy as np

# Carregar os dados de um arquivo FIF
epochs = mne.read_epochs('ssvep-epo.fif')

# Filtragem dos dados (1-30 Hz)
epochs = epochs.filter(1, 30).copy()

Reading c:\Users\eduar\OneDrive\Documentos\7° periodo-CM\Interfaces Não Convencionais\atvssvep\src\ssvep-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   15998.05 ms
        0 CTF compensation matrices available
Not setting metadata
50 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1691 samples (3.303 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [2]:
# Inicializar contadores de acertos e tentativas
correct = 0
total_tentativas = len(epochs)

for i in range(len(epochs)):
    psd = epochs[i].compute_psd(method='multitaper', fmin=3, fmax=13, verbose=False)
    freqs = psd.freqs
    psd_values = psd.get_data()  
    
    max_freqs = []
    for ch in range(psd_values.shape[0]):  
        max_idx = np.unravel_index(np.argmax(psd_values[ch], axis=None), psd_values[ch].shape)
        max_freq = freqs[max_idx[1]]
        max_freqs.append(max_freq)

    event_id = epochs[i].event_id
    evoked_freq = float(list(event_id.keys())[0])

    if any(np.isclose(max_freq, evoked_freq, atol=1.0) for max_freq in max_freqs):
        correct += 1


# Calcular a acurácia
accuracy = correct / total_tentativas * 100

# Exibir a acurácia
print(f'Acurácia da classificação: {accuracy:.2f}%')


Acurácia da classificação: 74.00%
